# 第 6 章：GRU 简化设计

> Gated Recurrent Unit - LSTM 的高效简化版本

---

## 本章目标

完成本章后，你将能够：

- [ ] 理解 GRU 与 LSTM 的关系和区别
- [ ] 掌握 GRU 的两个门：重置门和更新门
- [ ] 用 NumPy 从零实现 GRU
- [ ] 对比 GRU 和 LSTM 的性能和效率
- [ ] 学会在实践中选择 GRU 还是 LSTM

---

## GRU 简介

**GRU (Gated Recurrent Unit)** 由 Cho et al. 在 2014 年提出，是 LSTM 的简化版本：

| 特性 | LSTM | GRU |
|------|------|-----|
| 门的数量 | 3 个（遗忘、输入、输出） | 2 个（重置、更新） |
| 状态数量 | 2 个（h, c） | 1 个（h） |
| 参数量 | 4 × (H² + H×I + H) | 3 × (H² + H×I + H) |
| 训练速度 | 较慢 | 较快 |
| 性能 | 通常相当 | 通常相当 |

**设计理念**：用更少的参数达到相似的效果

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import time

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
torch.manual_seed(42)

print("环境准备完成！")

---

## 第一部分：GRU 结构详解

### 1.1 GRU 的两个门

**重置门 (Reset Gate) $r_t$**：
- 决定如何将新输入与之前的记忆结合
- $r_t \to 0$：忽略之前的隐状态
- $r_t \to 1$：完全使用之前的隐状态

**更新门 (Update Gate) $z_t$**：
- 决定保留多少之前的隐状态
- $z_t \to 0$：完全使用新计算的状态
- $z_t \to 1$：完全保留之前的状态

### 1.2 GRU 公式

$$r_t = \sigma(W_r \cdot [h_{t-1}, x_t] + b_r) \quad \text{(重置门)}$$

$$z_t = \sigma(W_z \cdot [h_{t-1}, x_t] + b_z) \quad \text{(更新门)}$$

$$\tilde{h}_t = \tanh(W_h \cdot [r_t \odot h_{t-1}, x_t] + b_h) \quad \text{(候选隐状态)}$$

$$h_t = (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \quad \text{(最终隐状态)}$$

In [ ]:
# 可视化：GRU 结构

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ========================================
# 左图：LSTM 结构（复杂）
# ========================================
ax = axes[0]
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)

# 主体
rect = plt.Rectangle((2, 2), 8, 6, fill=True, facecolor='lightyellow', 
                       edgecolor='orange', linewidth=2)
ax.add_patch(rect)
ax.text(6, 8.5, 'LSTM Cell', ha='center', fontsize=14, fontweight='bold')

# 三个门
gate_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
gate_names = ['遗忘门\n$f_t$', '输入门\n$i_t$', '输出门\n$o_t$']
gate_x = [3, 5.5, 8]

for x, color, name in zip(gate_x, gate_colors, gate_names):
    rect = plt.Rectangle((x-0.5, 4), 1, 1.5, fill=True, facecolor=color,
                          edgecolor='black', linewidth=1.5, alpha=0.7)
    ax.add_patch(rect)
    ax.text(x, 4.75, name, ha='center', va='center', fontsize=9)

# 细胞状态
ax.annotate('', xy=(10, 7), xytext=(2, 7),
            arrowprops=dict(arrowstyle='->', color='purple', lw=3))
ax.text(6, 7.3, '$c_t$', ha='center', fontsize=11, color='purple')

ax.text(6, 1.2, '3个门 + 细胞状态\n参数多，计算慢', ha='center', fontsize=11, color='orange')
ax.axis('off')

# ========================================
# 右图：GRU 结构（简化）
# ========================================
ax = axes[1]
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)

# 主体
rect = plt.Rectangle((2, 2), 8, 6, fill=True, facecolor='lightgreen', 
                       edgecolor='green', linewidth=2)
ax.add_patch(rect)
ax.text(6, 8.5, 'GRU Cell', ha='center', fontsize=14, fontweight='bold')

# 两个门
gate_colors = ['#FF6B6B', '#4ECDC4']
gate_names = ['重置门\n$r_t$', '更新门\n$z_t$']
gate_x = [4, 8]

for x, color, name in zip(gate_x, gate_colors, gate_names):
    rect = plt.Rectangle((x-0.7, 4), 1.4, 1.5, fill=True, facecolor=color,
                          edgecolor='black', linewidth=1.5, alpha=0.7)
    ax.add_patch(rect)
    ax.text(x, 4.75, name, ha='center', va='center', fontsize=10)

# 只有隐状态
ax.annotate('', xy=(10, 6.5), xytext=(2, 6.5),
            arrowprops=dict(arrowstyle='->', color='blue', lw=3))
ax.text(6, 6.8, '$h_t$ (唯一状态)', ha='center', fontsize=11, color='blue')

ax.text(6, 1.2, '2个门，无细胞状态\n参数少 25%，速度快', ha='center', fontsize=11, color='green')
ax.axis('off')

plt.tight_layout()
plt.show()

### 1.3 GRU 与 LSTM 的对应关系

GRU 可以看作是 LSTM 的简化，两者有对应关系：

| LSTM | GRU | 说明 |
|------|-----|------|
| 遗忘门 $f_t$ | 更新门 $z_t$ | GRU 的 $z_t$ 同时控制遗忘和输入 |
| 输入门 $i_t$ | $(1-z_t)$ | 与遗忘互补 |
| 输出门 $o_t$ | 无 | GRU 直接输出隐状态 |
| 细胞状态 $c_t$ | 无 | GRU 只有隐状态 $h_t$ |
| 候选 $\tilde{c}_t$ | 候选 $\tilde{h}_t$ | 由重置门控制 |

**关键简化**：
1. 合并遗忘门和输入门为一个更新门
2. 去掉细胞状态，只保留隐状态
3. 去掉输出门

In [ ]:
def explain_gru_gates():
    """
    解释 GRU 门的作用
    """
    print("=" * 60)
    print("GRU 门机制详解")
    print("=" * 60)
    
    explanation = """
    
┌─────────────────────────────────────────────────────────────┐
│                      重置门 (Reset Gate)                    │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  公式: r_t = σ(W_r · [h_{t-1}, x_t] + b_r)                  │
│                                                             │
│  作用: 控制如何利用之前的隐状态来计算候选状态                │
│                                                             │
│  r_t → 0: h̃_t = tanh(W_h · [0, x_t] + b_h)                 │
│           完全忽略之前的隐状态，就像"重新开始"              │
│                                                             │
│  r_t → 1: h̃_t = tanh(W_h · [h_{t-1}, x_t] + b_h)           │
│           完全使用之前的隐状态                              │
│                                                             │
│  应用场景: 当序列中出现新的主题或上下文转换时               │
│                                                             │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│                      更新门 (Update Gate)                   │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  公式: z_t = σ(W_z · [h_{t-1}, x_t] + b_z)                  │
│                                                             │
│  作用: 控制保留多少旧状态 vs 使用多少新计算的状态           │
│                                                             │
│  最终状态: h_t = (1-z_t) ⊙ h̃_t + z_t ⊙ h_{t-1}             │
│                   ↑                    ↑                    │
│                新内容               旧内容                  │
│                                                             │
│  z_t → 0: h_t ≈ h̃_t    (使用新状态)                        │
│  z_t → 1: h_t ≈ h_{t-1} (保持旧状态，信息直接传递)          │
│                                                             │
│  这正是解决梯度消失的关键！当 z_t→1 时，梯度可以直接传递    │
│                                                             │
└─────────────────────────────────────────────────────────────┘
"""
    
    print(explanation)
    
    # 可视化更新门的插值效果
    fig, ax = plt.subplots(figsize=(10, 5))
    
    h_prev = np.array([0.8, 0.2])  # 旧状态
    h_tilde = np.array([0.1, 0.9])  # 新候选
    
    z_values = np.linspace(0, 1, 5)
    colors = plt.cm.viridis(np.linspace(0, 1, len(z_values)))
    
    for z, color in zip(z_values, colors):
        h_t = (1 - z) * h_tilde + z * h_prev
        ax.scatter([h_t[0]], [h_t[1]], s=200, c=[color], 
                   label=f'z={z:.2f}', edgecolors='black', linewidth=1.5)
    
    # 标注端点
    ax.scatter([h_tilde[0]], [h_tilde[1]], s=300, marker='^', 
               c='green', edgecolors='black', linewidth=2, label='$\\tilde{h}_t$ (新)', zorder=10)
    ax.scatter([h_prev[0]], [h_prev[1]], s=300, marker='s', 
               c='red', edgecolors='black', linewidth=2, label='$h_{t-1}$ (旧)', zorder=10)
    
    # 连线
    ax.plot([h_tilde[0], h_prev[0]], [h_tilde[1], h_prev[1]], 
            'k--', alpha=0.3, linewidth=2)
    
    ax.set_xlabel('维度 1', fontsize=12)
    ax.set_ylabel('维度 2', fontsize=12)
    ax.set_title('更新门 z 的插值效果: $h_t = (1-z)\\tilde{h}_t + z \\cdot h_{t-1}$', fontsize=14)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-0.1, 1.0)
    ax.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.show()


explain_gru_gates()

---

## 第二部分：NumPy 从零实现 GRU

### 2.1 GRU Cell 实现

In [ ]:
class GRUCell:
    """
    GRU 单元的 NumPy 实现
    
    公式:
        r_t = sigmoid(W_r @ [h_{t-1}, x_t] + b_r)     # 重置门
        z_t = sigmoid(W_z @ [h_{t-1}, x_t] + b_z)     # 更新门  
        h_tilde = tanh(W_h @ [r_t * h_{t-1}, x_t] + b_h)  # 候选状态
        h_t = (1 - z_t) * h_tilde + z_t * h_{t-1}     # 最终状态
    
    参数:
        input_size: 输入特征维度
        hidden_size: 隐藏状态维度
    """
    
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # ========================================
        # 参数初始化
        # 将三个门的权重合并（r, z, h）
        # ========================================
        concat_size = hidden_size + input_size
        
        # Xavier 初始化
        scale = np.sqrt(2.0 / (concat_size + hidden_size))
        
        # 合并权重: (3 * hidden_size, hidden_size + input_size)
        # 顺序: [W_r, W_z, W_h]
        self.W = np.random.randn(3 * hidden_size, concat_size) * scale
        
        # 偏置: (3 * hidden_size,)
        self.b = np.zeros(3 * hidden_size)
    
    def sigmoid(self, x):
        """数值稳定的 Sigmoid"""
        return np.where(x >= 0,
                        1 / (1 + np.exp(-x)),
                        np.exp(x) / (1 + np.exp(x)))
    
    def forward(self, x_t, h_prev):
        """
        GRU 单步前向传播
        
        参数:
            x_t: 当前输入, shape: (input_size,)
            h_prev: 上一时刻隐状态, shape: (hidden_size,)
        
        返回:
            h_t: 当前隐状态, shape: (hidden_size,)
        """
        H = self.hidden_size
        
        # ========================================
        # Step 1: 拼接输入
        # ========================================
        concat = np.concatenate([h_prev, x_t])  # (H + I,)
        
        # ========================================
        # Step 2: 计算重置门和更新门
        # ========================================
        # 一次性计算 r 和 z 的线性变换
        gates_rz = self.W[0:2*H, :] @ concat + self.b[0:2*H]
        
        r_gate = self.sigmoid(gates_rz[0:H])    # 重置门, (H,)
        z_gate = self.sigmoid(gates_rz[H:2*H])  # 更新门, (H,)
        
        # ========================================
        # Step 3: 计算候选隐状态
        # 注意：这里使用 r_gate * h_prev 而不是原始 h_prev
        # ========================================
        # 重置后的隐状态
        h_reset = r_gate * h_prev  # (H,)
        
        # 拼接重置后的隐状态和输入
        concat_reset = np.concatenate([h_reset, x_t])  # (H + I,)
        
        # 候选隐状态
        h_tilde = np.tanh(self.W[2*H:3*H, :] @ concat_reset + self.b[2*H:3*H])  # (H,)
        
        # ========================================
        # Step 4: 更新隐状态（关键步骤！）
        # h_t = (1 - z_t) * h_tilde + z_t * h_{t-1}
        # 这是一个插值：z_t 决定新旧状态的混合比例
        # ========================================
        h_t = (1 - z_gate) * h_tilde + z_gate * h_prev
        
        # 保存中间值（用于可视化和调试）
        self.cache = {
            'r_gate': r_gate,
            'z_gate': z_gate,
            'h_tilde': h_tilde,
            'h_prev': h_prev,
            'h_t': h_t
        }
        
        return h_t


# 测试 GRU Cell
print("=" * 60)
print("GRU Cell 测试")
print("=" * 60)

input_size = 4
hidden_size = 8

gru_cell = GRUCell(input_size, hidden_size)

# 初始状态
h_0 = np.zeros(hidden_size)

# 输入
x = np.random.randn(input_size)

# 前向传播
h_1 = gru_cell.forward(x, h_0)

print(f"\n输入维度: {input_size}")
print(f"隐藏维度: {hidden_size}")
print(f"\n输出 h_1 形状: {h_1.shape}")
print(f"\n各门的平均值:")
print(f"  重置门 r: {gru_cell.cache['r_gate'].mean():.4f}")
print(f"  更新门 z: {gru_cell.cache['z_gate'].mean():.4f}")

### 2.2 完整 GRU 类

In [ ]:
class GRU:
    """
    完整的 GRU 网络
    """
    
    def __init__(self, input_size, hidden_size):
        self.cell = GRUCell(input_size, hidden_size)
        self.hidden_size = hidden_size
    
    def forward(self, X, h_0=None):
        """
        处理整个序列
        
        参数:
            X: 输入序列, shape: (seq_len, input_size)
            h_0: 初始隐状态, shape: (hidden_size,)
        
        返回:
            h_all: 所有时间步的隐状态, shape: (seq_len, hidden_size)
            h_last: 最后的隐状态, shape: (hidden_size,)
        """
        seq_len = X.shape[0]
        
        if h_0 is None:
            h_0 = np.zeros(self.hidden_size)
        
        h_t = h_0
        h_all = []
        gate_history = {'r': [], 'z': []}
        
        for t in range(seq_len):
            h_t = self.cell.forward(X[t], h_t)
            h_all.append(h_t)
            gate_history['r'].append(self.cell.cache['r_gate'].mean())
            gate_history['z'].append(self.cell.cache['z_gate'].mean())
        
        self.gate_history = gate_history
        
        return np.array(h_all), h_t


# 测试
seq_len = 20
gru = GRU(input_size, hidden_size)

X = np.random.randn(seq_len, input_size)
h_all, h_last = gru.forward(X)

print(f"输入序列形状: {X.shape}")
print(f"所有隐状态形状: {h_all.shape}")

# 可视化门激活
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

ax = axes[0]
ax.plot(gru.gate_history['r'], 'r-', linewidth=2, label='重置门 r')
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('门激活值', fontsize=11)
ax.set_title('GRU 门激活随时间变化', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

ax = axes[1]
ax.plot(gru.gate_history['z'], 'b-', linewidth=2, label='更新门 z')
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('时间步', fontsize=11)
ax.set_ylabel('门激活值', fontsize=11)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

### 2.3 与 PyTorch 对比验证

In [ ]:
def compare_gru_with_pytorch():
    """
    验证 NumPy GRU 实现的正确性
    """
    print("=" * 60)
    print("NumPy GRU vs PyTorch GRU 对比")
    print("=" * 60)
    
    seq_len = 5
    input_size = 3
    hidden_size = 4
    
    np.random.seed(42)
    torch.manual_seed(42)
    
    # PyTorch GRU
    gru_pt = nn.GRUCell(input_size, hidden_size)
    
    # NumPy GRU
    gru_np = GRUCell(input_size, hidden_size)
    
    # 复制权重
    # PyTorch GRUCell 权重顺序: [r, z, n] (reset, update, new)
    # weight_ih: (3H, I), weight_hh: (3H, H)
    W_ih = gru_pt.weight_ih.detach().numpy()  # (3H, I)
    W_hh = gru_pt.weight_hh.detach().numpy()  # (3H, H)
    
    # 拼接成 (3H, H+I)，注意顺序是 [h, x]
    gru_np.W = np.concatenate([W_hh, W_ih], axis=1)
    gru_np.b = (gru_pt.bias_ih.detach().numpy() + 
                gru_pt.bias_hh.detach().numpy())
    
    # 输入
    X_np = np.random.randn(seq_len, input_size).astype(np.float32)
    X_pt = torch.from_numpy(X_np)
    
    # NumPy 前向
    h_np = np.zeros(hidden_size, dtype=np.float32)
    h_np_all = []
    
    for t in range(seq_len):
        h_np = gru_np.forward(X_np[t], h_np)
        h_np_all.append(h_np.copy())
    
    # PyTorch 前向
    h_pt = torch.zeros(1, hidden_size)
    h_pt_all = []
    
    for t in range(seq_len):
        h_pt = gru_pt(X_pt[t:t+1], h_pt)
        h_pt_all.append(h_pt.detach().numpy().flatten().copy())
    
    # 比较
    h_np_all = np.array(h_np_all)
    h_pt_all = np.array(h_pt_all)
    
    diff = np.abs(h_np_all - h_pt_all)
    
    print(f"\n最大差异: {diff.max():.2e}")
    print(f"平均差异: {diff.mean():.2e}")
    
    if diff.max() < 1e-5:
        print("\n✓ NumPy GRU 实现正确！")
    else:
        print("\n注意: 存在一些差异（可能是由于 PyTorch 的实现细节）")


compare_gru_with_pytorch()

---

## 第三部分：GRU vs LSTM 对比实验

### 3.1 参数量对比

In [ ]:
def compare_parameters():
    """
    比较 LSTM 和 GRU 的参数量
    """
    print("=" * 60)
    print("参数量对比: LSTM vs GRU")
    print("=" * 60)
    
    input_sizes = [32, 64, 128, 256]
    hidden_sizes = [64, 128, 256, 512]
    
    results = []
    
    for input_size, hidden_size in zip(input_sizes, hidden_sizes):
        # LSTM 参数: 4 个门，每个门有 W_ih, W_hh, b_ih, b_hh
        # 总参数 = 4 * (H*I + H*H + H + H) = 4 * (H*I + H*H + 2H)
        lstm_params = 4 * (hidden_size * input_size + hidden_size * hidden_size + 2 * hidden_size)
        
        # GRU 参数: 3 个门，同样结构
        # 总参数 = 3 * (H*I + H*H + H + H) = 3 * (H*I + H*H + 2H)
        gru_params = 3 * (hidden_size * input_size + hidden_size * hidden_size + 2 * hidden_size)
        
        reduction = (lstm_params - gru_params) / lstm_params * 100
        
        results.append({
            'input': input_size,
            'hidden': hidden_size,
            'lstm': lstm_params,
            'gru': gru_params,
            'reduction': reduction
        })
        
        print(f"\nInput={input_size}, Hidden={hidden_size}:")
        print(f"  LSTM: {lstm_params:,} 参数")
        print(f"  GRU:  {gru_params:,} 参数")
        print(f"  减少: {reduction:.1f}%")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 5))
    
    x = np.arange(len(results))
    width = 0.35
    
    lstm_params = [r['lstm']/1000 for r in results]  # 转为千
    gru_params = [r['gru']/1000 for r in results]
    
    bars1 = ax.bar(x - width/2, lstm_params, width, label='LSTM', color='coral')
    bars2 = ax.bar(x + width/2, gru_params, width, label='GRU', color='steelblue')
    
    ax.set_xlabel('配置 (Input, Hidden)', fontsize=12)
    ax.set_ylabel('参数量 (千)', fontsize=12)
    ax.set_title('LSTM vs GRU 参数量对比', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels([f"({r['input']}, {r['hidden']})" for r in results])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 添加减少比例标注
    for i, r in enumerate(results):
        ax.text(i, max(lstm_params[i], gru_params[i]) + 50, 
                f'-{r["reduction"]:.0f}%', ha='center', fontsize=10, color='green')
    
    plt.tight_layout()
    plt.show()
    
    print("\n结论: GRU 比 LSTM 减少约 25% 的参数量")
    
    return results


param_results = compare_parameters()

### 3.2 训练速度对比

In [ ]:
def compare_speed():
    """
    比较 LSTM 和 GRU 的训练速度
    """
    print("=" * 60)
    print("训练速度对比: LSTM vs GRU")
    print("=" * 60)
    
    # 配置
    batch_size = 32
    seq_len = 100
    input_size = 64
    hidden_size = 128
    n_iterations = 100
    
    # 创建模型
    lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
    gru = nn.GRU(input_size, hidden_size, batch_first=True)
    
    # 准备数据
    X = torch.randn(batch_size, seq_len, input_size)
    
    # 测试 LSTM
    start = time.time()
    for _ in range(n_iterations):
        out, _ = lstm(X)
        out.sum().backward()
        lstm.zero_grad()
    lstm_time = time.time() - start
    
    # 测试 GRU
    start = time.time()
    for _ in range(n_iterations):
        out, _ = gru(X)
        out.sum().backward()
        gru.zero_grad()
    gru_time = time.time() - start
    
    speedup = lstm_time / gru_time
    
    print(f"\n配置: batch={batch_size}, seq_len={seq_len}, hidden={hidden_size}")
    print(f"迭代次数: {n_iterations}")
    print(f"\nLSTM 时间: {lstm_time:.3f}s")
    print(f"GRU 时间:  {gru_time:.3f}s")
    print(f"GRU 加速: {speedup:.2f}x")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(8, 5))
    
    models = ['LSTM', 'GRU']
    times = [lstm_time, gru_time]
    colors = ['coral', 'steelblue']
    
    bars = ax.bar(models, times, color=colors, edgecolor='black', linewidth=2)
    
    ax.set_ylabel('时间 (秒)', fontsize=12)
    ax.set_title(f'前向+反向传播时间 ({n_iterations} 次迭代)', fontsize=14)
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, t in zip(bars, times):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
                f'{t:.2f}s', ha='center', fontsize=12)
    
    ax.text(0.5, max(times) * 0.7, f'GRU 快 {speedup:.1f}x', 
            ha='center', fontsize=14, color='green',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    return lstm_time, gru_time


lstm_time, gru_time = compare_speed()

### 3.3 长期记忆任务对比

In [ ]:
def compare_memory_task():
    """
    在长期记忆任务上对比 LSTM 和 GRU
    """
    print("=" * 60)
    print("长期记忆任务: LSTM vs GRU")
    print("=" * 60)
    
    def generate_data(n_samples, seq_len):
        """生成数据：记住第一个位置的符号"""
        X = np.random.randn(n_samples, seq_len, 1) * 0.1
        Y = np.zeros((n_samples, 1))
        
        for i in range(n_samples):
            label = np.random.choice([-1, 1])
            X[i, 0, 0] = label
            Y[i, 0] = label
        
        return torch.FloatTensor(X), torch.FloatTensor(Y)
    
    class MemoryModel(nn.Module):
        def __init__(self, model_type, hidden_size):
            super().__init__()
            if model_type == 'LSTM':
                self.rnn = nn.LSTM(1, hidden_size, batch_first=True)
            else:
                self.rnn = nn.GRU(1, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, 1)
        
        def forward(self, x):
            out, _ = self.rnn(x)
            return torch.tanh(self.fc(out[:, -1, :]))
    
    # 测试配置
    seq_lengths = [10, 30, 50, 100, 150]
    hidden_size = 32
    n_train = 500
    n_test = 100
    n_epochs = 100
    
    results = {'LSTM': [], 'GRU': []}
    
    for seq_len in seq_lengths:
        print(f"\n序列长度 = {seq_len}")
        
        X_train, Y_train = generate_data(n_train, seq_len)
        X_test, Y_test = generate_data(n_test, seq_len)
        
        for model_type in ['LSTM', 'GRU']:
            torch.manual_seed(42)
            model = MemoryModel(model_type, hidden_size)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
            criterion = nn.MSELoss()
            
            # 训练
            for epoch in range(n_epochs):
                model.train()
                optimizer.zero_grad()
                output = model(X_train)
                loss = criterion(output, Y_train)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            # 测试
            model.eval()
            with torch.no_grad():
                pred = model(X_test)
                acc = ((pred > 0) == (Y_test > 0)).float().mean().item()
            
            results[model_type].append(acc)
            print(f"  {model_type}: {acc*100:.1f}%")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(len(seq_lengths))
    width = 0.35
    
    lstm_accs = [a * 100 for a in results['LSTM']]
    gru_accs = [a * 100 for a in results['GRU']]
    
    bars1 = ax.bar(x - width/2, lstm_accs, width, label='LSTM', color='coral', edgecolor='black')
    bars2 = ax.bar(x + width/2, gru_accs, width, label='GRU', color='steelblue', edgecolor='black')
    
    ax.axhline(y=50, color='gray', linestyle='--', label='随机猜测', alpha=0.7)
    
    ax.set_xlabel('序列长度', fontsize=12)
    ax.set_ylabel('准确率 (%)', fontsize=12)
    ax.set_title('长期记忆任务: LSTM vs GRU', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels([str(s) for s in seq_lengths])
    ax.legend()
    ax.set_ylim(0, 105)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\n结论: LSTM 和 GRU 在长期记忆任务上表现相当")
    
    return results


memory_results = compare_memory_task()

---

## 第四部分：实践指南

### 4.1 何时选择 GRU vs LSTM

In [ ]:
def selection_guide():
    """
    GRU vs LSTM 选择指南
    """
    print("=" * 70)
    print("GRU vs LSTM 选择指南")
    print("=" * 70)
    
    guide = """
    
┌────────────────────────────────────────────────────────────────────┐
│                         选择 GRU 当:                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  ✓ 训练数据相对较少（GRU 参数少，不易过拟合）                      │
│  ✓ 需要更快的训练/推理速度                                         │
│  ✓ 计算资源有限（移动端、嵌入式）                                  │
│  ✓ 序列长度适中（不是特别长）                                      │
│  ✓ 初次尝试，想要更简单的模型                                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│                         选择 LSTM 当:                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  ✓ 训练数据充足                                                    │
│  ✓ 任务需要非常长的依赖（如很长的文档）                            │
│  ✓ 模型精度优先于速度                                              │
│  ✓ 已有大量 LSTM 相关的预训练模型/经验                             │
│  ✓ 需要更精细的信息流控制（三个独立的门）                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│                         实践建议                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  1. 默认从 GRU 开始尝试（更快，更简单）                            │
│  2. 如果 GRU 性能不够，尝试 LSTM                                   │
│  3. 两者都试试，选择表现更好的那个                                 │
│  4. 考虑使用 Transformer（现在更主流）                             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│                    常见任务的推荐选择                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  语音识别:     LSTM（传统选择）或 Transformer（现代选择）          │
│  机器翻译:     Transformer >> LSTM/GRU                             │
│  时间序列:     GRU（快速实验）→ LSTM（优化）                       │
│  文本分类:     GRU 通常足够                                        │
│  命名实体识别: BiLSTM-CRF（经典）或 BERT（现代）                   │
│  音乐生成:     LSTM（需要长期结构）                                │
│  手写识别:     GRU（速度重要）或 LSTM                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
"""
    
    print(guide)


selection_guide()

### 4.2 PyTorch 使用示例

In [ ]:
def pytorch_usage_examples():
    """
    PyTorch 中 GRU 的使用示例
    """
    print("=" * 60)
    print("PyTorch GRU 使用示例")
    print("=" * 60)
    
    # 示例 1: 基本 GRU
    print("\n1. 基本 GRU 使用:")
    print("-" * 40)
    
    batch_size = 4
    seq_len = 10
    input_size = 8
    hidden_size = 16
    
    gru = nn.GRU(input_size, hidden_size, batch_first=True)
    x = torch.randn(batch_size, seq_len, input_size)
    
    output, h_n = gru(x)
    
    print(f"输入 x: {x.shape}")
    print(f"输出 output: {output.shape}  # 所有时间步的隐状态")
    print(f"最终状态 h_n: {h_n.shape}  # 最后一个时间步的隐状态")
    
    # 示例 2: 多层双向 GRU
    print("\n2. 多层双向 GRU:")
    print("-" * 40)
    
    num_layers = 2
    bidirectional = True
    
    gru_bi = nn.GRU(input_size, hidden_size, 
                    num_layers=num_layers,
                    bidirectional=bidirectional,
                    batch_first=True)
    
    output_bi, h_n_bi = gru_bi(x)
    
    print(f"配置: {num_layers} 层, 双向={bidirectional}")
    print(f"输出 output: {output_bi.shape}  # hidden_size * 2 (双向)")
    print(f"最终状态 h_n: {h_n_bi.shape}  # num_layers * 2 (双向)")
    
    # 示例 3: 用于分类任务
    print("\n3. GRU 文本分类模型:")
    print("-" * 40)
    
    class GRUClassifier(nn.Module):
        def __init__(self, vocab_size, embed_dim, hidden_size, num_classes):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embed_dim)
            self.gru = nn.GRU(embed_dim, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, num_classes)
        
        def forward(self, x):
            # x: (batch, seq_len) - 词索引
            embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
            output, h_n = self.gru(embedded)  # h_n: (1, batch, hidden)
            h_n = h_n.squeeze(0)  # (batch, hidden)
            return self.fc(h_n)  # (batch, num_classes)
    
    model = GRUClassifier(vocab_size=10000, embed_dim=128, 
                          hidden_size=256, num_classes=5)
    
    # 假设输入
    x_text = torch.randint(0, 10000, (batch_size, seq_len))
    logits = model(x_text)
    
    print(f"模型结构: Embedding → GRU → Linear")
    print(f"输入 x: {x_text.shape} (词索引)")
    print(f"输出 logits: {logits.shape} (分类 logits)")
    print(f"\n总参数量: {sum(p.numel() for p in model.parameters()):,}")


pytorch_usage_examples()

---

## 本章总结

### 核心要点

1. **GRU 是 LSTM 的简化版本**：
   - 2 个门（重置门、更新门）vs LSTM 的 3 个门
   - 1 个状态（h）vs LSTM 的 2 个状态（h, c）
   - 参数减少约 25%

2. **GRU 公式**：
   ```
   r_t = σ(W_r · [h_{t-1}, x_t] + b_r)       # 重置门
   z_t = σ(W_z · [h_{t-1}, x_t] + b_z)       # 更新门
   h̃_t = tanh(W_h · [r_t ⊙ h_{t-1}, x_t] + b_h)  # 候选状态
   h_t = (1 - z_t) ⊙ h̃_t + z_t ⊙ h_{t-1}    # 最终状态
   ```

3. **选择建议**：
   - 默认从 GRU 开始（更快、更简单）
   - 如果需要更高精度，尝试 LSTM
   - 现代任务考虑 Transformer

### 代码模板

```python
# PyTorch GRU
gru = nn.GRU(input_size, hidden_size, 
             num_layers=2, 
             bidirectional=True,
             batch_first=True)

output, h_n = gru(input_seq)
```

---

## 下一章预告

**第 7 章：双向与多层 RNN**

如何进一步增强 RNN 的表达能力？
- 双向 RNN：同时利用过去和未来的上下文
- 多层 RNN：增加网络深度
- 组合使用：BiLSTM、多层 GRU 等

👉 [07_bidirectional_multilayer.ipynb](./07_bidirectional_multilayer.ipynb)